In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer2DTo3D(nn.Module):
    def __init__(self, input_dim=21, output_dim=31, d_model=128, nhead=8, num_encoder_layers=6, dim_feedforward=512):
        super(Transformer2DTo3D, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward),
            num_layers=num_encoder_layers
        )
        self.fc = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, output_dim)
        )

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x = self.embedding(x)  # (batch_size, seq_len, d_model)
        x = x.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, d_model)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Aggregate sequence for regression
        x = self.fc(x)
        return x

# Save model definition to use in inference
print("Model definition loaded.")


In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Dummy dataset (Replace this with actual 2D input and 3D target data)
input_data = torch.randn(1000, 10, 21)  # 1000 samples, sequence length 10, 21 features
target_data = torch.randn(1000, 31)     # Corresponding 3D outputs

# Dataloader
dataset = TensorDataset(input_data, target_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, optimizer, loss
model = Transformer2DTo3D(input_dim=21, output_dim=31)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for batch in dataloader:
        x, y = batch
        optimizer.zero_grad()
        predictions = model(x)
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

# Save model weights
torch.save(model.state_dict(), "transformer_2d_to_3d_v2.pth")
print("Model training completed and weights saved to 'transformer_2d_to_3d_v2.pth'.")


In [ ]:
# Import necessary libraries
import torch

# Load the model
model = Transformer2DTo3D(input_dim=21, output_dim=31)
model.load_state_dict(torch.load("transformer_2d_to_3d_v2.pth"))
model.eval()
print("Model weights loaded for inference.")

# Inference function
def run_inference(model, input_data):
    """
    Runs inference using the trained model.
    
    Args:
        model (Transformer2DTo3D): Trained model.
        input_data (torch.Tensor): 2D input data (batch_size, seq_len, input_dim).
        
    Returns:
        torch.Tensor: Predicted 3D outputs.
    """
    with torch.no_grad():
        predictions = model(input_data)
    return predictions

# Example inference (Replace `test_input` with actual test data)
test_input = torch.randn(5, 10, 21)  # 5 samples, sequence length 10, 21 features
predictions = run_inference(model, test_input)
print("Predictions:", predictions)
